In [27]:
pip install pymodbus

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [33]:
from pymodbus.client import ModbusTcpClient
import struct
import time

In [39]:
# Configuração do CR310 (Modbus TCP Server)
HOST = "localhost"  # IP do CR310 na rede
PORT = 502              # Porta padrão Modbus TCP
SLAVE_ID = 1            # ID do escravo Modbus
ADDRESSES = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22]   # Endereços dos registradores iniciais para cada variável
SENSOR_NAMES = ["Temperatura Interna", "Estado da Bateria", "Temperatura do Ar", "Umidade do Ar", "Temperatura do Módulo", "Velocidade do Vento", "Radiação Global", "Radiação Difusa", "Radiação Difusa Corrigida", "Radiação Difusa Calculada", "Angulo Solar Azimute", "Elevação Solar"]
INTERVAL = 1            # Tempo entre leituras (segundos)

# Criando o cliente Modbus
client = ModbusTcpClient(HOST, port=PORT)

In [41]:
try:
    if client.connect():
        print("Conectado ao CR310 via Modbus TCP")

        while True:
            for i, addr in enumerate(ADDRESSES):
                response = client.read_holding_registers(address=addr, count=2, slave=SLAVE_ID)

                if response.isError():
                    print(f"Erro ao ler {SENSOR_NAMES[i]} (endereço {addr}):", response)
                else:
                    # Convertendo os registros lidos em bytes (Big-Endian)
                    registers = response.registers
                    raw_bytes = struct.pack('>2H', *registers)

                    # Convertendo para float de 32 bits (IEEE 754, Float Trocado)
                    value = struct.unpack('<f', raw_bytes)[0]  # "<f" -> Little Endian (troca de bytes)
                    print(f"{SENSOR_NAMES[i]}: {value}")

            time.sleep(INTERVAL)  # Aguarda antes da próxima leitura
    else:
        print("Falha ao conectar ao CR310")
except KeyboardInterrupt:
    print("\nEncerrando...")
finally:
    client.close()

Connection to (localhost, 502) failed: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente


Falha ao conectar ao CR310
